# CpGPTGrimAge3

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [77]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya
import numpy as np

## Instantiate model class

In [78]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.CpGPTGrimAge3)

class CpGPTGrimAge3(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        """
        Scales an array based on the median and standard deviation.
        """
        median = torch.tensor(self.preprocess_dependencies[0], device=x.device, dtype=x.dtype)
        std = torch.tensor(self.preprocess_dependencies[1], device=x.device, dtype=x.dtype)
        x = (x - median) / std
        return x

    def postprocess(self, x):
        """
        Converts from a Cox parameter to age in units of years.
        """
        cox_mean = self.postprocess_dependencies[0]
        cox_std = self.postprocess_dependencies[1]
        age_mean = self.postprocess_dependencies[2]
        age_std = self.postprocess_dependencies[3]

        # Normalize
        x = (x - cox_mean) / cox_std

        # Scale
        x = (x * age_std) + age_mean

        return x



In [79]:
model = pya.models.CpGPTGrimAge3()

## Define clock metadata

In [80]:
model.metadata["clock_name"] = 'cpgptgrimage3'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2025
model.metadata["approved_by_author"] = '✅'
model.metadata["citation"] = "de Lima Camillo, Lucas Paulo, et al. \"CpGPT: a foundation model for DNA methylation.\" bioRxiv (2024): 2024-10."
model.metadata["doi"] = "https://doi.org/10.1101/2024.10.24.619766"
model.metadata["research_only"] = True
model.metadata["notes"] = None

## Download clock dependencies

In [81]:
logger = pya.logger.Logger()
urls = [
    "https://pyaging.s3.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/cpgpt_grimage3_weights.csv",
    "https://pyaging.s3.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/cpgpt_grimage3_weights_all_datasets.csv",
    "https://pyaging.s3.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/cpgpt_pcgrimage3_weights.csv",
    "https://pyaging.s3.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/input_scaler_mean.npy",
    "https://pyaging.s3.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/input_scaler_scale.npy",
    "https://pyaging.s3.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/input_scaler_mean_all_datasets.npy",
    "https://pyaging.s3.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/input_scaler_scale_all_datasets.npy",
    "https://pyaging.s3.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/pca_scaler_mean.npy",
    "https://pyaging.s3.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/pca_scaler_scale.npy",
    "https://pyaging.s3.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/cpgpt_pcgrimage3_pca_components.npy",
]
dir = "."
for url in urls:
    pya.utils.download(url, dir, logger, indent_level=1)

|-----------> Downloading data to ./cpgpt_grimage3_weights.csv
|-----------> in progress: 100.0000%
|-----------> Downloading data to ./cpgpt_grimage3_weights_all_datasets.csv
|-----------> in progress: 100.0000%
|-----------> Downloading data to ./cpgpt_pcgrimage3_weights.csv
|-----------> in progress: 100.0000%
|-----------> Downloading data to ./input_scaler_mean.npy
|-----------> in progress: 100.0000%%
|-----------> Downloading data to ./input_scaler_scale.npy
|-----------> in progress: 100.0000%%
|-----------> Downloading data to ./input_scaler_mean_all_datasets.npy
|-----------> in progress: 100.0000%%
|-----------> Downloading data to ./input_scaler_scale_all_datasets.npy
|-----------> in progress: 100.0000%%
|-----------> Downloading data to ./pca_scaler_mean.npy
|-----------> in progress: 100.0000%%
|-----------> Downloading data to ./pca_scaler_scale.npy
|-----------> in progress: 100.0000%%
|-----------> Downloading data to ./cpgpt_pcgrimage3_pca_components.npy
|-----------

## Load features

#### From CSV

In [82]:
df = pd.read_csv('cpgpt_grimage3_weights_all_datasets.csv')
model.features = df['feature'].tolist()

In [83]:
df.head()

,feature,coefficient
0,age,0.736548
1,grimage2timp1,0.137387
2,grimage2packyrs,0.287951
3,grimage2logcrp,0.244555
4,grimage2b2m,0.054020


## Load weights into base model

#### Linear model

In [84]:
weights = torch.tensor(df['coefficient'].tolist()).unsqueeze(0)
intercept = torch.tensor([0.0])

#### Linear model

In [85]:
base_model = pya.models.LinearModel(input_dim=len(model.features))

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

In [86]:
scale_mean = np.load('input_scaler_mean_all_datasets.npy')
scale_std = np.load('input_scaler_scale_all_datasets.npy')

model.reference_values = None

## Load preprocess and postprocess objects

In [87]:
model.preprocess_name = 'scale'
model.preprocess_dependencies = [scale_mean, scale_std]

In [88]:
model.postprocess_name = 'cox_to_years'
# model.postprocess_dependencies = [
#     -6.60427912e-16,
#     1.72004470842156,
#     58.8234007654456,
#     13.091231557630831
# ]

model.postprocess_dependencies = [
    -0.0203713448,
    1.59202528637141,
    65.10638128157404,
    12.363204038137225
]

## Check all clock parameters

In [89]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '✅',
 'citation': 'de Lima Camillo, Lucas Paulo, et al. "CpGPT: a foundation model '
             'for DNA methylation." bioRxiv (2024): 2024-10.',
 'clock_name': 'cpgptgrimage3',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.1101/2024.10.24.619766',
 'notes': None,
 'research_only': True,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2025}
reference_values: None
preprocess_name: 'scale'
preprocess_dependencies: [array([ 6.52416417e+01,  3.48958664e+04,  1.79132118e+01,  3.93809992e-01,
        1.73345822e+06,  3.49194003e+02,  9.49228459e+03,  9.06203720e+02,
       -5.17726985e-01, -2.51474184e-01, -7.81861920e-03, -2.22291158e-01,
       -2.95438692e-01, -1.72762763e-01, -2.27222377e-01,  1.04146839e-01,
        1.35866795e-01, -2.30935616e-01, -5.11943990e-01,  2.09570292e-01,
       -3.75342389e-02, -2.584

## Basic test

In [90]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float).double()
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[-141.7864],
        [ -83.2373],
        [ 158.5985],
        [  70.5253],
        [ 106.9326],
        [-138.7387],
        [ -15.3592],
        [-293.0767],
        [ -20.1291],
        [ 239.3673]], dtype=torch.float64, grad_fn=<AddBackward0>)

## Save torch model

In [91]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [92]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: input_scaler_mean_all_datasets.npy
Deleted file: cpgpt_pcgrimage3_pca_components.npy
Deleted file: cpgpt_pcgrimage3_weights.csv
Deleted file: input_scaler_scale_all_datasets.npy
Deleted file: input_scaler_scale.npy
Deleted file: pca_scaler_mean.npy
Deleted file: cpgpt_grimage3_weights_all_datasets.csv
Deleted file: pca_scaler_scale.npy
Deleted file: cpgpt_grimage3_weights.csv
Deleted file: input_scaler_mean.npy
